# S02T04: Geração dos pickles com os dados aumentados 

## The one with the libs

In [1]:
import librosa
import tensorflow_io as tfio
import glob
import re #regex
import numpy as np
from IPython.display import Image
import pickle
from scipy import signal
from tqdm import tqdm

## The one with mel-spectrogram 201 x 201

In [2]:
stft_datas = []

In [ ]:
def compute_mel(audioname):
    wav = librosa.load(audioname, sr=22050, duration=3.5)[0]
    melspec = librosa.feature.melspectrogram(wav,sr=22050,n_fft=1024,hop_length=385,n_mels=201)
    logmel = librosa.core.power_to_db(melspec)
            
         # Créditos: Jederson
    max_pad_len = 201
    pad_width = max_pad_len - logmel.shape[1]
    logmel = np.pad(logmel, pad_width=((0, 0), (0, pad_width)), mode='constant')

    return logmel

In [3]:
def compute_stft(audioname):
         
        wav = librosa.load(audioname, sr=22050, duration=3.5)[0]
        stftspec = np.abs(librosa.stft(wav, n_fft=400,  hop_length=385, window=signal.blackmanharris(400)))
         
        logstft = librosa.core.power_to_db(stftspec**2)
        
        stft_datas.append(stftspec)
        
        
        return logstft

In [4]:
def compute_harmonic(id_):
    kernel_size=31
    power=2.0
    mask=False
    margin=1.0

    harmspec, P = librosa.decompose.hpss(stft_datas[id_], kernel_size=kernel_size, power=power, mask=mask, margin=margin)

    logharm = librosa.core.power_to_db(harmspec)
   

    return logharm

## The one with time and frequency transforms

In [5]:
def compute_transforms(spec, vet):
        params = vet
        freq_transforms = []
        time_transforms = []
        for i in params:
            freq_mask = tfio.experimental.audio.freq_mask(spec, param=i).numpy().astype('float32')
            time_mask = tfio.experimental.audio.time_mask(spec, param=i).numpy().astype('float32')
            freq_transforms.append(freq_mask)
            time_transforms.append(time_mask)
        return freq_transforms, time_transforms

## The one with the audio name and class 

In [6]:
def get_audio_name_class(path):
    audio_name = path.split('/')[-1]
#     print (audio_name)
    audio_name_wo_ext = re.sub(r'\.wav', "", audio_name)
#     print (audio_name_wo_ext)
    label = path.split('/')[-2]
#     print (label)
    return audio_name_wo_ext, label

## The one who saves the specs

In [7]:
x_harm = []
y_harm = []
    
def run(dataset_path, pickle_name):
    
    #Captura os nomes das pastas
    name_folders = glob.glob(dataset_path + '*')
#     x_mel = []
#     y_mel = []
    
    
    for names_ in name_folders:
        
        print(names_)

        #Captura o caminho completo do audio
        path_audios = glob.glob(names_ + '/*.wav')
        
        for i,path_ in enumerate(tqdm(path_audios)):
            
            audio_name, label = get_audio_name_class(path_)
#             logmel = compute_mel(path_)
            logstft = compute_stft(path_)
            logharm = compute_harmonic(i)
            
            
#             x_mel.append(logmel)
#             y_mel.append(label)
            
            x_harm.append(logharm)
            y_harm.append(label)
            
            #Estas são as classes que necessitaram fazer um aumento de dados, dependendo da quantidade
            # necessaria maior o parametro de aumento
            
            if label == 'Pasodoble' or label == 'Slowwaltz':
                label_seq = [label]*8
#                 freq,time = compute_transforms(logmel,[15,20,25,30])
#                 x_mel.extend(freq)
#                 x_mel.extend(time)                  
#                 y_mel.extend(label_seq)
                
                freq_harm,time_harm = compute_transforms(logharm,[15,20,25,30])
                x_harm.extend(freq_harm)
                x_harm.extend(time_harm)                  
                y_harm.extend(label_seq)
                
            elif label == 'Viennesewaltz':
                label_seq = [label]
#                 freq,time = compute_transforms(logmel,[15])
#                 x_mel.extend(freq)
# #                 x_mel.extend(time)                  
#                 y_mel.extend(label_seq)
                
                freq_harm,time_harm = compute_transforms(logharm,[15])
                x_harm.extend(freq_harm)
#                 x_harm.extend(time_harm)                  
                y_harm.extend(label_seq)
    
            elif label == 'Salsa':
                label_seq = [label]*10
#                 freq,time = compute_transforms(logmel,[15,20,25,30,35])
#                 x_mel.extend(freq)
#                 x_mel.extend(time)                  
#                 y_mel.extend(label_seq)
                
                freq_harm,time_harm = compute_transforms(logharm,[15,20,25,30,35])
                x_harm.extend(freq_harm)
                x_harm.extend(time_harm)                  
                y_harm.extend(label_seq)
                
            elif label == 'Wcswing':
                label_seq = [label]*22
#                 freq,time = compute_transforms(logmel,[15,20,25,30,35,40,45,50,55,60,65])
#                 x_mel.extend(freq)
#                 x_mel.extend(time)                  
#                 y_mel.extend(label_seq)
                
                freq_harm,time_harm = compute_transforms(logharm,[15,20,25,30,35,40,45,50,55,60,65])
                x_harm.extend(freq_harm)
                x_harm.extend(time_harm)                  
                y_harm.extend(label_seq)
    
    
                
#     pickle.dump(x_mel, open('x_' + str(pickle_name) +'_aug_logmelspec.pickle', 'wb'))
#     pickle.dump(y_mel, open('y_' + str(pickle_name) +'_aug_logmelspec.pickle', 'wb'))
    

In [9]:
stft_datas=[]

In [10]:
pickle.dump(x_harm, open('x_' +'treino_aug_logharmspec2.pickle', 'wb'))
pickle.dump(y_harm, open('y_' + 'treino_aug_logharmspec2.pickle', 'wb'))

## The one who runs the script

In [8]:
run('/home/arlaxad/Documentos/Sistemas de Informação/2020.3/base_de_dados/extendedballroom_v1 (cópia).1(.wav)/Treino/', 'treino') 

  0%|          | 0/1716 [00:00<?, ?it/s]

/home/arlaxad/Documentos/Sistemas de Informação/2020.3/base_de_dados/extendedballroom_v1 (cópia).1(.wav)/Treino/Samba


  0%|          | 0/950 [00:00<?, ?it/s]

/home/arlaxad/Documentos/Sistemas de Informação/2020.3/base_de_dados/extendedballroom_v1 (cópia).1(.wav)/Treino/Viennesewaltz


  0%|          | 0/74 [00:00<?, ?it/s]

...

  0%|          | 0/1717 [00:00<?, ?it/s]

/home/arlaxad/Documentos/Sistemas de Informação/2020.3/base_de_dados/extendedballroom_v1 (cópia).1(.wav)/Treino/Waltz


  0%|          | 0/1674 [00:00<?, ?it/s]

/home/arlaxad/Documentos/Sistemas de Informação/2020.3/base_de_dados/extendedballroom_v1 (cópia).1(.wav)/Treino/Tango


100%|██████████| 1674/1674 [08:09<00:00,  3.42it/s]


## The one who checks the pickles

In [ ]:
def read_pickle(name):
    with (open(name, 'rb')) as openfile:
        while True:
            try:
                one_instance = pickle.load(openfile)
            except EOFError:
                break
    one_instance = np.asanyarray(one_instance)
    return one_instance

In [ ]:
# Dimensões sem augmentation - x:(6971, 173, 173) e y:(6971,)
x_train_mel = read_pickle('x_train_aug_logmelspec.pickle')
print (x_train.shape)
y_train_mel = read_pickle('y_train_aug_logmelspec.pickle')
print (y_train.shape)
x_train_harm = read_pickle('x_train_aug_logmelspec.pickle')
print (x_train.shape)
y_train_harm = read_pickle('y_train_aug_logmelspec.pickle')
print (y_train.shape)

# Como ficou as amostras por classes com o Aumentation

In [ ]:
y_train_mel.count("classes")
y_train_harm.count("classes")